# 1. Initialize SparkContext

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CreditCardFraud_RDD_Based").master("local[*]").getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/05/09 10:05:45 WARN Utils: Your hostname, HuuNghia-PC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/09 10:05:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 10:05:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 2. Load & Clean Data

In [ ]:
lines = sc.textFile("hdfs:///hcmus/22120227/Exercises/Lab3/data/creditcard.csv")
header = lines.first()
data = lines.filter(lambda row: row != header)

# Remove exact duplicates
data = data.distinct()
print(f"Rows after deduplication: {data.count()}")

# Remove malformed rows
data = data.filter(lambda line: len(line.split(',')) == 31)
print(f"Records after cleaning: {data.count()}")

Rows after deduplication: 283726
Records after cleaning: 283726


# 3. Parse into (label, raw-features) tuples

In [12]:
def parse_to_pair(line):
    vals = line.strip().replace('"','').split(',')
    raw_vals = [float(v) for v in vals]
    features = [ float(vals[i]) for i in range(1,29) ] + [ float(vals[0]), float(vals[29]) ]
    label = float(vals[30])
    return (label, features, raw_vals)

parsed = data.map(parse_to_pair)
print("Parsed:", parsed.first())

Parsed: (0.0, [-0.425965884412454, 0.960523044882985, 1.14110934232219, -0.168252079760302, 0.42098688077219, -0.0297275516639742, 0.476200948720027, 0.260314333074874, -0.56867137571251, -0.371407196834471, 1.34126198001957, 0.359893837038039, -0.358090652573631, -0.137133700217612, 0.517616806555742, 0.401725895589603, -0.0581328233640131, 0.0686531494425432, -0.0331937877876282, 0.0849676720682049, -0.208253514656728, -0.559824796253248, -0.0263976679795373, -0.371426583174346, -0.232793816737034, 0.105914779097957, 0.253844224739337, 0.0810802569229443, 2.0, 3.67], [2.0, -0.425965884412454, 0.960523044882985, 1.14110934232219, -0.168252079760302, 0.42098688077219, -0.0297275516639742, 0.476200948720027, 0.260314333074874, -0.56867137571251, -0.371407196834471, 1.34126198001957, 0.359893837038039, -0.358090652573631, -0.137133700217612, 0.517616806555742, 0.401725895589603, -0.0581328233640131, 0.0686531494425432, -0.0331937877876282, 0.0849676720682049, -0.208253514656728, -0.55982

# 4. Standardize each feature dimension

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

# Extract just the feature-vectors
feat_rdd = parsed.map(lambda lp: Vectors.dense(lp[1]))

# Fit a StandardScalerModel (with mean & std)
scaler = StandardScaler(withMean=True, withStd=True).fit(feat_rdd)

# Transform every feature-vector
scaled_feat_rdd = scaler.transform(feat_rdd)

# Zip labels back onto the scaled features
rdd_lp = parsed.zip(scaled_feat_rdd).map(lambda x: (LabeledPoint(x[0][0], x[1]), x[0][2]))
print("First scaled LabeledPoint:", rdd_lp.first())

First scaled LabeledPoint: (LabeledPoint(0.0, [-0.2217028945454752,0.5858116621248715,0.7552925575140171,-0.11687713221997206,0.3043985479000141,-0.0214636285494593,0.38642519236408757,0.2215069801184997,-0.5176440604458157,-0.34370490530557013,1.3164167007570957,0.36253934785450453,-0.3603409172432784,-0.1442804456221721,0.564627350784502,0.45847037481254743,-0.06920175394664986,0.08017648079189918,-0.04048486903719246,0.11010679473848713,-0.287167055780041,-0.772630472830473,-0.04264187474139357,-0.6136466426018291,-0.4461864250142345,0.21940606321886716,0.6369806464936855,0.24550735753670333,-1.9967772787560036,-0.3386696403466281]), [2.0, -0.425965884412454, 0.960523044882985, 1.14110934232219, -0.168252079760302, 0.42098688077219, -0.0297275516639742, 0.476200948720027, 0.260314333074874, -0.56867137571251, -0.371407196834471, 1.34126198001957, 0.359893837038039, -0.358090652573631, -0.137133700217612, 0.517616806555742, 0.401725895589603, -0.0581328233640131, 0.0686531494425432, 

# 5. Split into train/test sets

In [5]:
train_rdd, test_rdd = rdd_lp.randomSplit([0.8, 0.2], seed=42)
print(f"Train size: {train_rdd.count()}, Test size: {test_rdd.count()}")

Train size: 227339, Test size: 56387


In [6]:
# Extract only the LabeledPoint objects from train_rdd
train_lp = train_rdd.map(lambda x: x[0])

# 6. Train & Tune Models

## 6.1. Prepare the evaluation function

In [7]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

def evaluate_rdd(preds_and_labels):
    # Compute ROC-AUC and PR-AUC
    metrics = BinaryClassificationMetrics(preds_and_labels)
    roc_auc = metrics.areaUnderROC
    pr_auc = metrics.areaUnderPR

    # Threshold at 0.5 to get hard predictions
    pl = preds_and_labels.map(lambda sl: ((1.0 if sl[0] >= 0.5 else 0.0), sl[1]))
    
    # Confusion matrix counts
    tp = pl.filter(lambda x: x == (1.0, 1.0)).count()
    fp = pl.filter(lambda x: x == (1.0, 0.0)).count()
    fn = pl.filter(lambda x: x == (0.0, 1.0)).count()
    tn = pl.filter(lambda x: x == (0.0, 0.0)).count()
    
    # Classic matrics
    accuracy  = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    
    return {"Accuracy": accuracy, "Precision": precision,
            "Recall": recall, "F1": f1,
            "ROC-AUC": roc_auc, "PR-AUC": pr_auc,
            "ConfusionMatrix": [[tn, fp], [fn, tp]]}

## 6.2. LBFGS Hyperparameter Experiments

In [8]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

# Define grid
iters_list = [50, 100, 200]

best_lbfgs = {"f1": 0.0, "iters": None}

# Loop over grid and evaluate
for iters in iters_list:
    model = LogisticRegressionWithLBFGS.train(
        train_lp,
        iterations=iters,
        numClasses=2
    )
    model.clearThreshold()
    preds = test_rdd.map(lambda p: (model.predict(p[0].features), p[0].label))
    res = evaluate_rdd(preds)
    print(f"LBFGS(iters={iters}) → PR‑AUC={res['PR-AUC']:.4f}, F1={res['F1']:.4f}")
    if res["F1"] > best_lbfgs["f1"]:
        best_lbfgs.update({"f1": res["F1"], "iters": iters})

# Print best configuration
print("\nBest LBFGS configuration:")
print(f"iterations = {best_lbfgs['iters']}")
print(f"F1 = {best_lbfgs['f1']:.4f}")

25/05/09 10:06:12 WARN Instrumentation: [63a266f1] Initial coefficients will be ignored! Its dimensions (1, 30) did not match the expected size (1, 30)
25/05/09 10:06:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
/home/huunghia/pyspark-env/lib/python3.12/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


LBFGS(iters=50) → PR‑AUC=0.4004, F1=0.7816


25/05/09 10:06:36 WARN Instrumentation: [e2722bd1] Initial coefficients will be ignored! Its dimensions (1, 30) did not match the expected size (1, 30)
/home/huunghia/pyspark-env/lib/python3.12/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


LBFGS(iters=100) → PR‑AUC=0.4004, F1=0.7816


25/05/09 10:06:56 WARN Instrumentation: [c30acf81] Initial coefficients will be ignored! Its dimensions (1, 30) did not match the expected size (1, 30)


LBFGS(iters=200) → PR‑AUC=0.4004, F1=0.7816

Best LBFGS configuration:
iterations = 50
F1 = 0.7816


## 6.3. SGD Hyperparameter Experiments

In [9]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

# Define grid
iters_list = [50, 100, 200]
step_list  = [10, 15, 20]

best_sgd = {"f1": 0.0, "iterations": None, "step": None}

# Loop over grid and evaluate
for iters in iters_list:
    for step in step_list:
        model = LogisticRegressionWithSGD.train(
            train_lp,
            iterations=iters,
            step=step,
            miniBatchFraction=0.5
        )
        model.clearThreshold()
        preds = test_rdd.map(lambda p: (model.predict(p[0].features), p[0].label))
        res   = evaluate_rdd(preds)
        print(f"SGD(iters={iters}, step={step}) → PR‑AUC={res['PR-AUC']:.4f}, F1={res['F1']:.4f}")
        
        if res["F1"] > best_sgd["f1"]:
            best_sgd.update({"f1": res["F1"], "iterations": iters, "step": step})

# Print best configuration
print("\nBest SGD configuration:")
print(f"iterations = {best_sgd['iterations']}")
print(f"step = {best_sgd['step']}")
print(f"F1 = {best_sgd['f1']:.4f}")

/home/huunghia/pyspark-env/lib/python3.12/site-packages/pyspark/mllib/classification.py:395: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  warnings.warn(
25/05/09 10:07:19 WARN GradientDescent: Testing against a convergenceTol when using miniBatchFraction < 1.0 can be unstable because of the stochasticity in sampling.


SGD(iters=50, step=10) → PR‑AUC=0.7180, F1=0.0524


25/05/09 10:07:39 WARN GradientDescent: Testing against a convergenceTol when using miniBatchFraction < 1.0 can be unstable because of the stochasticity in sampling.


SGD(iters=50, step=15) → PR‑AUC=0.7194, F1=0.0692


25/05/09 10:08:03 WARN GradientDescent: Testing against a convergenceTol when using miniBatchFraction < 1.0 can be unstable because of the stochasticity in sampling.


...

SGD(iters=200, step=10) → PR‑AUC=0.7200, F1=0.0802


25/05/09 10:10:05 WARN GradientDescent: Testing against a convergenceTol when using miniBatchFraction < 1.0 can be unstable because of the stochasticity in sampling.


SGD(iters=200, step=15) → PR‑AUC=0.7231, F1=0.0830


25/05/09 10:10:28 WARN GradientDescent: Testing against a convergenceTol when using miniBatchFraction < 1.0 can be unstable because of the stochasticity in sampling.


SGD(iters=200, step=20) → PR‑AUC=0.7227, F1=0.0828

Best SGD configuration:
iterations = 100
step = 20
F1 = 0.1509


## 6.4. Model Selection
In fraud detection, we prioritize **F1‑score** to balance catching as many fraudulent transactions as possible (recall) while minimizing false alarms (precision).  

From our hyperparameter tuning:  
- **LBFGS** achieved **F1 ≈ 0.7816**  
- **SGD** achieved **F1 ≈ 0.1509**  

Therefore, we select the **LBFGS** model for final evaluation.

# 7. Final Retraining and Evaluation on Test Set
Retrain the best models found in Section 7 on the full training RDD, then evaluate each on the held‑out test RDD.

In [ ]:
best_lbfgs_model = LogisticRegressionWithLBFGS.train(train_lp, iterations=50, numClasses=2)

# Extract only the LabeledPoint objects from the test RDD
test_lp = test_rdd.map(lambda x: x[0])  # x = (LabeledPoint, raw_vals)

# Extract true labels
test_labels = test_lp.map(lambda p: p.label)

# Predict using the trained model
best_lbfgs_model.clearThreshold()
preds = test_lp.map(lambda p: best_lbfgs_model.predict(p.features))

# Zip predictions and true labels
preds_lbfgs = preds.zip(test_labels)

# Evaluate using binary classification metrics
res_lbfgs = evaluate_rdd(preds_lbfgs)

# Print evaluation results
print("Final LBFGS Evaluation:")
for metric, value in res_lbfgs.items():
    print(f"  {metric:12} = {value}")

25/05/09 10:10:54 WARN Instrumentation: [82e10172] Initial coefficients will be ignored! Its dimensions (1, 30) did not match the expected size (1, 30)


Final LBFGS Evaluation:
  Accuracy     = 0.9993260857999184
  Precision    = 0.7727272726394628
  Recall       = 0.7906976743266632
  F1           = 0.7816091903131195
  ROC-AUC      = 0.907388567182293
  PR-AUC       = 0.4003566751256275
  ConfusionMatrix = [[56281, 20], [18, 68]]


# 8. Export Full Test Data with Predictions

In [ ]:
# Zip predictions with original raw values
test_raw_vals = test_rdd.map(lambda x: x[1])  # raw features from the CSV
test_preds = preds.map(lambda p: int(round(p)))  # Convert prediction to 0 or 1
test_joined = test_raw_vals.zip(test_preds)

# Define header: all original columns + prediction
header = ["Time"] + [f"V{i}" for i in range(1, 29)] + ["Amount", "Class", "prediction"]
header_line = ",".join(header)

# Convert each row to CSV line
data_lines = test_joined.map(lambda row: ",".join([str(x) for x in row[0]] + [str(row[1])])).collect()

# Save locally with header first
output_path = "results.csv"
with open(output_path, "w") as f:
    f.write(header_line + "\n")
    for line in data_lines:
        f.write(line + "\n")

print(f"Prediction file saved to: {output_path}")

Prediction file saved to: predictions_test.csv
